In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
start = timeit.default_timer()
# pybamm.set_logging_level("INFO")

In [2]:
def electrolyte_conductivity_Valoen2005(c_e, T):
    # T = T + 273.15
    # mol/m3 to molar
    c_e = c_e / 1000
    # mS/cm to S/m
    return (1e-3 / 1e-2) * (
        c_e
        * (
            (-10.5 + 0.0740 * T - 6.96e-5 * T ** 2)
            + c_e * (0.668 - 0.0178 * T + 2.80e-5 * T ** 2)
            + c_e ** 2 * (0.494 - 8.86e-4 * T)
        )
        ** 2
    )
def electrolyte_diffusivity_Valoen2005(c_e, T):
    # T = T + 273.15
    # mol/m3 to molar
    c_e = c_e / 1000

    T_g = 229 + 5 * c_e
    D_0 = -4.43 - 54 / (T - T_g)
    D_1 = -0.22

    # cm2/s to m2/s
    # note, in the Valoen paper, ln means log10, so its inverse is 10^x
    return (10 ** (D_0 + D_1 * c_e)) * 1e-4

In [3]:
model = pybamm.lithium_ion.DFN()
param = pybamm.ParameterValues("Chen2020")
c_e = model.variables["Electrolyte concentration [mol.m-3]"]
T = model.variables["Cell temperature [K]"]
D_e = model.param.D_e_dimensional
kappa_e = model.param.kappa_e_dimensional
model.variables["Electrolyte diffusivity [m2.s-1]"] = D_e(c_e, T)
model.variables["Electrolyte conductivity [S.m-1]"] = kappa_e(c_e, T)
# use electrolyte Nyman_polynomial
V_max = 4.2;        V_min = 2.5; 
exp_RPT_text = [ (
    f"Discharge at 1 C until {V_min} V", #  (1 minute period)  
    "Rest for 1 hours (5 minute period)",  
    f"Charge at 1 C until {V_max} V",
    "Rest for 1 hours (5 minute period)",
    ) ]
Experiment_Breakin= pybamm.Experiment( exp_RPT_text * 1     )
sim1 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    #var_pts=var_pts,
)
solution1 = sim1.solve()

stop = timeit.default_timer()
print('running time: ' + str(stop - start) +'s')

two-step
two-step
running time: 5.059748200001195s


In [4]:
# now go for Valoen:
param.update({"Electrolyte conductivity [S.m-1]": electrolyte_conductivity_Valoen2005})
param.update({"Electrolyte diffusivity [m2.s-1]": electrolyte_diffusivity_Valoen2005})
sim2 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
    #var_pts=var_pts,
)
solution_Valoen = sim2.solve()

two-step


In [5]:
# now go for Landesfied 3:7:
def electrolyte_conductivity_base_Landesfeind2019(c_e, T, coeffs):
    c = c_e / 1000  # mol.m-3 -> mol.l
    p1, p2, p3, p4, p5, p6 = coeffs
    A = p1 * (1 + (T - p2))
    B = 1 + p3 * pybamm.sqrt(c) + p4 * (1 + p5 * pybamm.exp(1000 / T)) * c
    C = 1 + c**4 * (p6 * pybamm.exp(1000 / T))
    sigma_e = A * c * B / C  # mS.cm-1

    return sigma_e / 10

def electrolyte_diffusivity_base_Landesfeind2019(c_e, T, coeffs):
    c = c_e / 1000  # mol.m-3 -> mol.l
    p1, p2, p3, p4 = coeffs
    A = p1 * pybamm.exp(p2 * c)
    B = pybamm.exp(p3 / T)
    C = pybamm.exp(p4 * c / T)
    D_e = A * B * C * 1e-10  # m2/s

    return D_e
def electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019(c_e, T):
    coeffs = np.array([1.01e3, 1.01, -1.56e3, -4.87e2])

    return electrolyte_diffusivity_base_Landesfeind2019(c_e, T, coeffs)

def electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019(c_e, T):
    coeffs = np.array([5.21e-1, 2.28e2, -1.06, 3.53e-1, -3.59e-3, 1.48e-3])

    return electrolyte_conductivity_base_Landesfeind2019(c_e, T, coeffs)

param.update({"Electrolyte conductivity [S.m-1]": electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019})
param.update({"Electrolyte diffusivity [m2.s-1]": electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019})
sim3 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
    #var_pts=var_pts,
)
solution_Landes = sim3.solve()

two-step


In [6]:
# now go for constant:
param.update({"Electrolyte conductivity [S.m-1]": 0.7})
param.update({"Electrolyte diffusivity [m2.s-1]": 3e-10})
sim4 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
    #var_pts=var_pts,
)
solution_Constant= sim4.solve()

two-step


In [7]:
label = ["Nyman Poly","Landes Exp","Constant","Valoen Poly"] 
var = [
    "Terminal voltage [V]",
    "Electrolyte concentration [mol.m-3]",
    "Electrolyte diffusivity [m2.s-1]",
    "Electrolyte conductivity [S.m-1]"
]
quick_plot = pybamm.QuickPlot(
    [
    solution1.cycles[0].steps[0],
    solution_Landes.cycles[0].steps[0],
    solution_Constant.cycles[0].steps[0],
    solution_Valoen.cycles[0].steps[0],
    ], var,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=3560.24089793284, step=35.6024089793284), Ou…